In [ ]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#importing tensorflow
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten,BatchNormalization,MaxPooling2D
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import mnist

In [ ]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
X = X_train
y = y_train

test_x = X_test

X = X / 255.0
test_x = test_x / 255.0

In [ ]:
X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)

In [ ]:
y = to_categorical(y)

print(f"Label size {y.shape}")

Label size (60000, 10)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)


In [ ]:
batch_size = 64

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images



train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
import datetime
!rm -rf ./logs/ 
#tf.reset_default_graph()
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
def scheduler(epoch):
  if epoch >= 40 and epoch <50:
    return 0.001
  elif (epoch >= 60 and epoch <80):
    return 0.001
  
  elif (epoch >= 81 and epoch <99):
    return 0.001

  else:
    return 0.01

In [ ]:
filepath="weights.hdf5"
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.datasets import mnist

checkpoint_1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

In [ ]:
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
   

model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform'))
   

model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
   

model.add(Flatten())
model.add(Dense(13, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
	
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 8)           1160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)               

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.001),metrics=["accuracy"])

In [ ]:
for i in range(50):
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
  callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
  history = model.fit(train_gen, 
                              epochs = 300,
                              #use_multiprocessing=True, 
                              #workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1]
                              
                              )

Epoch 1/300
937/937 [==============================] - 47s 50ms/step - loss: 0.0497 - accuracy: 0.9850 - val_loss: 0.0459 - val_accuracy: 0.9861

Epoch 00001: val_accuracy did not improve from 0.98706
Epoch 1/300
937/937 [==============================] - 47s 50ms/step - loss: 0.0478 - accuracy: 0.9851 - val_loss: 0.0504 - val_accuracy: 0.9842

Epoch 00001: val_accuracy did not improve from 0.98706
Epoch 1/300
937/937 [==============================] - 47s 50ms/step - loss: 0.0466 - accuracy: 0.9856 - val_loss: 0.0519 - val_accuracy: 0.9842

Epoch 00001: val_accuracy did not improve from 0.98706
Epoch 1/300
937/937 [==============================] - 47s 50ms/step - loss: 0.0451 - accuracy: 0.9857 - val_loss: 0.0532 - val_accuracy: 0.9845

Epoch 00001: val_accuracy did not improve from 0.98706
Epoch 1/300
937/937 [==============================] - 47s 50ms/step - loss: 0.0469 - accuracy: 0.9854 - val_loss: 0.0572 - val_accuracy: 0.9852

Epoch 00001: val_accuracy did not improve from 0.9